This code amalgamates the summary information from the 'mylocalschool.wales' website into one large dictionary for easy access. 

In [1]:
import requests
from bs4 import BeautifulSoup

This is a list of the schools in the local area that we want to focus on.

In [20]:
list_of_schools = ['Severn Primary','St. Mary\'s R.C. Primary School','St.Paul\'s C/W Primary School','Grangetown Primary School','St Patrick\'s R C School','Ysgol Gynradd Gymraeg Hamadryad','Ninian Park Primary School','Kitchener Primary School','Radnor Primary School','Lansdowne Primary School','Ysgol Treganna','Ysgol Gymraeg Pwll Coch']

This function makes a dictionary which associates each school name to the URL of its summary page.

In [21]:
def make_school_URL_dict(schools):
    toppage = requests.get("http://mylocalschool.wales.gov.uk/Schools/SchoolSearch?lang=en")
    soup = BeautifulSoup(toppage.content, 'html.parser')
    school_links = soup.select("a[href*=/School]")[2:]
    schools = [list(school.children)[0] for school in school_links]
    school_URLs = ["http://mylocalschool.wales.gov.uk" + school['href'] for school in school_links]
    schoolURL_dict = {}
    for s,school in enumerate(schools):
        schoolURL_dict[school] = school_URLs[s]
    return schoolURL_dict

In [22]:
schoolURL_dict = make_school_URL_dict(list_of_schools)

This next function takes the URL of a schools summary page and makes a dictionary listing the stats for that school.

In [42]:
def make_stat_dict(schoolURL):
    school_pages = requests.get(schoolURL)
    school_soup = BeautifulSoup(school_pages.content, 'html.parser')
    summary = school_soup.find_all('div',id=False, class_="summaryBox")
    stat_dict = {}
    for i in range(len(summary)):
        stat = ''.join(char for char in list(summary[i].children)[1].getText() if char.isalnum() or char=='.')
        if stat.isnumeric(): stat=float(stat)
        stat_name = (''.join(char for char in list(summary[i].children)[3].getText() if char.isalnum() or char==' ')).strip()
        if '%' in list(summary[i].children)[1].getText(): stat_name += "(%)"
        stat_dict[stat_name] = stat
    return stat_dict

This dictionary links each school to the summary dictionary for that school.

In [43]:
the_grand_dict = {}
for school in list_of_schools:
    the_grand_dict[school] = make_stat_dict(schoolURL_dict[school])

In [44]:
the_grand_dict

{'Grangetown Primary School': {'Attendance during the year Primary only(%)': '94.2',
  'Free school meals FSM  3 year average Primary only(%)': '27.2',
  'Number of Pupils 2017': 404.0,
  'Pupil Teacher Ratio PTR Primary only': '19.6',
  'Pupils achieving the expected outcome in the Foundation Phase areas of learning(%)': '76.3',
  'Pupils who have reached the expected level  Core subject indicator Key Stage 2(%)': 80.0,
  'School budget per pupil': 3623.0,
  'Support Category': 'Yellow'},
 'Kitchener Primary School': {'Attendance during the year Primary only(%)': '94.5',
  'Average number of minutes per week allocated for curricular PE Primary only': 105.0,
  'Free school meals FSM  3 year average Primary only(%)': '29.9',
  'Number of Pupils 2017': 486.0,
  'Pupil Teacher Ratio PTR Primary only': '20.5',
  'Pupils achieving the expected outcome in the Foundation Phase areas of learning(%)': '80.8',
  'Pupils who have reached the expected level  Core subject indicator Key Stage 2(%)':